In [1]:
import numpy as np
import pandas as pd
import pyodbc

In [21]:
years = list(range(89, 99))+ [99] + list(range(1400, 1403))
years = [1403]
for year in years:
    # Path to your .accdb file
    file_path = fr'C:\Users\Mahsa\Desktop\geo\raw_data\{year}\Outfile-{year}.accdb'
    # Connection string (for 64-bit Access Database Engine)
    conn_str = (
        r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
        f'DBQ={file_path};'
    )
    # Connect
    conn = pyodbc.connect(conn_str)
    # List all table names
    tables = [table.table_name for table in conn.cursor().tables(tableType='TABLE')]
    print(f"Year is :{str(year)}".center(50))
    #---------------------------------------------------------------------------------
    Rdata = pd.read_sql(f"SELECT * FROM R{year}P1", conn)
    Udata = pd.read_sql(f"select * from U{year}P1", conn)
    data = pd.concat([Rdata, Udata], axis=0)
    
    data.rename(
        columns = {
            "Address": "HHID",
            "DYCOL01": "IndiNum",
            "DYCOL03": "RelationHead",
            "DYCOL04": "Sex",
            "DYCOL05": "Age",
            "DYCOL06": "Literacy",
            "DYCOL07": "Current_student",
            "DYCOL08": "EducationLevel",
            "DYCOL09": "EmploymentStatus",
            "DYCOL10": "MaritalState"   
        },
        inplace= True
    )
    data["HHID"] = data["HHID"].astype("str")
    mask = data["HHID"].str.contains(r"\.")
    data.loc[mask, 'HHID'] = data.loc[mask, "HHID"].str.split(".").str[0]
    data = data.loc[data["HHID"].notna()]
    
    data["IndiNum"] = data["IndiNum"].astype("str")
    mask = data["IndiNum"].str.contains(r"\.")
    data.loc[mask, 'IndiNum'] = data.loc[mask, "IndiNum"].str.split(".").str[0]
    data["IndiNum"] = data["IndiNum"].str.pad(2,'left',"0")
    
    data["EducationLevel"] = pd.to_numeric(data["EducationLevel"], errors='coerce')
    data["RelationHead"] = pd.to_numeric(data["RelationHead"], errors='coerce')
    data["Sex"] = pd.to_numeric(data["Sex"], errors='coerce')
    data["Age"] = pd.to_numeric(data["Age"], errors='coerce')
    data["Literacy"] = pd.to_numeric(data["Literacy"], errors='coerce')
    data["Current_student"] = pd.to_numeric(data["Current_student"], errors='coerce')
    data["EmploymentStatus"] = pd.to_numeric(data["EmploymentStatus"], errors='coerce')
    data["MaritalState"] = pd.to_numeric(data["MaritalState"], errors='coerce')
    
    data["RelationHead"] = np.where(
        data["RelationHead"] == 1, "Head",np.where(
            data["RelationHead"] == 2, "Spouse", np.where(
                data["RelationHead"] == 3, "Child", np.where(
                    data["RelationHead"] == 4, "s/d in low", np.where(
                        data["RelationHead"] == 5, "Grandchild", np.where(
                            data["RelationHead"] == 6, "Mom/Dad", np.where(
                                data["RelationHead"] == 7, "Bro/Sis", np.where(
                                    data["RelationHead"] == 8, "OtherRelative", np.where(
                                        data["RelationHead"] == 9, "NoneRe", pd.NA
                                    )
                                )
                            )
                        )
                    )
                )
            )
        )
    )
    data["Sex"] = np.where(
        data["Sex"]==1, "Male", np.where(
            data["Sex"] == 2, "Female", pd.NA
        )
    )
    data["Literacy"] = np.where(
        data["Literacy"] == 1, "Literate", np.where(
            data["Literacy"] == 2, "Illiterate", pd.NA
        )
    )
    data["Current_student"] = np.where(
        data["Current_student"]== 1, "Studying", np.where(
            data["Current_student"] == 2, "NotStudying", pd.NA
        )
    )
    data["EmploymentStatus"] = np.where(
        data["EmploymentStatus"] == 1, "Employed", np.where(
            data["EmploymentStatus"]== 2, "Unemployed", np.where(
                data["EmploymentStatus"]== 3, "IncomeWithoutJob", np.where(
                    data["EmploymentStatus"]==4, "Student", np.where(
                        data["EmploymentStatus"]== 5, "Housekeeper", np.where(
                            data["EmploymentStatus"]== 6, "Other", pd.NA
                        )
                    )
                )
            )
        )
    )
    data["MaritalState"] = np.where(
        data["MaritalState"]==1, "Married", np.where(
            data["MaritalState"]==2, "Widow", np.where(
                data["MaritalState"]==3, "Divorced", np.where(
                    data["MaritalState"]==4, "NotMarried", pd.NA
                )
            )
        )
    )
    
    print(
        data.apply(lambda col: col.unique()),
        f"The number of Rows is : {data.shape[0]}",       
    )
    print(data.loc[data["Literacy"].isna(),"Age"].max()<7)
    print(data.loc[(data["Current_student"].isna()) & (data["Literacy"].notna()),"Literacy"].nunique()==1)
    print(
    ((data["Literacy"].isna()) | (data["Literacy"] == "Illiterate")).sum() == data["Current_student"].isna().sum()
    )
    data.to_pickle(fr"C:\Users\Mahsa\Desktop\geo\base2\P1_{year}")
    

                  Year is :1403                   


C:\Users\Mahsa\AppData\Local\Temp\ipykernel_16476\3153833754.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Rdata = pd.read_sql(f"SELECT * FROM R{year}P1", conn)
C:\Users\Mahsa\AppData\Local\Temp\ipykernel_16476\3153833754.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Udata = pd.read_sql(f"select * from U{year}P1", conn)


HHID                [20603452450, 20817487553, 20603452443, 214035...
IndiNum             [04, 05, 01, 02, 03, 06, 07, 08, 09, 10, 11, 1...
RelationHead        [Child, Head, Spouse, Mom/Dad, Bro/Sis, s/d in...
Sex                                              [Female, Male, <NA>]
Age                 [4, 1, 35, 33, 10, 5, 38, 31, 15, 9, 2, 44, 34...
Literacy                                 [<NA>, Literate, Illiterate]
Current_student                         [<NA>, NotStudying, Studying]
EducationLevel      [nan, 4.0, 1.0, 2.0, 5.0, 6.0, 3.0, 9.0, 7.0, ...
EmploymentStatus    [<NA>, Employed, Housekeeper, Student, Unemplo...
MaritalState             [<NA>, Married, NotMarried, Widow, Divorced]
dtype: object The number of Rows is : 121118
True
True
True


In [22]:
years = list(range(89, 99))+ [99] + list(range(1400, 1403))
years = [1403]
for year in years:
    df = pd.read_pickle(f"P1_{year}")
    if year == 93:
        df.loc[(df["HHID"]=="10207048807") & (df["IndiNum"]=='02'),'RelationHead']="Spouse"
    df["NumChild_U5"] = np.where(
        (df["RelationHead"]=="Child") & (df["Age"]<5), 1,0
    )
    df["NumChild_B5_7"] = np.where(
        (df["RelationHead"]=="Child") & ((df["Age"]>=5) & (df["Age"]<7)), 1,0
    )
    df["NumChild_B7_18"] = np.where(
        (df["RelationHead"]=="Child") & ((df["Age"]>=7) & (df["Age"]<18)), 1,0
    )
    df["NumChildOver18"] = np.where(
        (df["RelationHead"]=="Child") & (df["Age"]>=18), 1,0
    )
    df["HeadWoman"] = np.where(
        (df["IndiNum"]=="01") & (df["RelationHead"]=="Head") & (df["Sex"]=="Female"), 1, 0
    )
    df["Dep_Student"] = np.where(
        (df["NumChild_B7_18"]==1) & (df["Current_student"]=="NotStudying"), 1, 0
    )
    df["Num_Literate"] = np.where(
        df["Literacy"]=="Literate", 1, 0
    )
    df["NumSpouse"] = np.where(
        df["RelationHead"]=="Spouse", 1, 0
    )
    df["NumEmployed"] = np.where(
        df["EmploymentStatus"]=="Employed", 1, 0
    )
    df["NumOld"] = np.where(
        df["Age"]>= 65, 1, 0
    )
    df["NumMale"] = np.where(
        df["Sex"]=="Male", 1, 0
    )
    df["NumNotfirstR"] = np.where(
        df["RelationHead"].isin(['Head', 'Spouse', 'Child']), 0, 1
    )
    dfagg = df.groupby("HHID").aggregate(
        HHID = ("HHID", "first"),
        HHsize = ("HHID", "count"),
        AveAge = ("Age","mean"),
        NumChild_U5 = ("NumChild_U5","sum"),
        NumChild_B5_7 = ("NumChild_B5_7","sum"),
        NumChild_B7_18 = ("NumChild_B7_18","sum"),
        NumChildOver18 = ('NumChildOver18','sum'),
        HeadWoman = ("HeadWoman","sum"),
        Dep_Student = ('Dep_Student','sum'),
        Num_Literate = ('Num_Literate','sum'),
        NumSpouse = ('NumSpouse','sum'),
        NumEmployed = ('NumEmployed','sum'),
        NumOld = ('NumOld','sum'),
        NumMale = ('NumMale','sum'),
        NumNotfirstR = ("NumNotfirstR", "sum")
    ).reset_index(drop = True)
    df["MaleP"] = dfagg["NumMale"]/ dfagg["HHsize"]
    print(f'\n\n------------{year}------------\n')
    print(dfagg["HeadWoman"].max() ==1)
    print(dfagg["NumSpouse"].max())
    print(np.all(dfagg["NumChild_U5"]+dfagg["NumChild_B5_7"]+dfagg["NumChild_B7_18"]+dfagg["NumChildOver18"]+dfagg["NumNotfirstR"]+dfagg["NumSpouse"]+1==dfagg["HHsize"]))
    dfagg.to_pickle(fr"HHP1_{year}")




------------1403------------

True
2
True
